# 3. PHD Filtering
The Probability Hypothesis Density (PHD) filter is a recursive, multiple-target tracker which estimates the number of targets and their states, given a set of observations. This notebook demonstrates this filter and the consequences of changing the detection probabily of the linear sensor.

The PHD filter implemented in this notebook is the Gaussian-Mixture PHD filter. This filter provides a closed-form solution to the PHD recursion algorithm, or approximated provided some assumptions about the dynamic systems are stipulated:

- The targets are independend from each other and are Gaussian or Poisson distributed
- The measured clutter is also independent of the target measurements
- The target states are dependent on the multi-target prior

We shall modify the example tutorial from [2] to observe the effects of detection probability in the performance of the PHD filter.

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from datetime import datetime
from phdsupplement import gen_ground_truths, gen_detectors, run_tracker

Let us generate some paths using the following parameters

In [2]:
# Parameters
start_time = datetime.now()
birth_probability = 0.2
death_probability = 0.005
number_steps = 20

truths, truths_by_time, start_truths, transition_model = gen_ground_truths(start_time, birth_probability, death_probability, number_steps)

# Then generate path detections
detection_probability = 0.9
clutter_rate = 3.0
all_measurements, measurement_model = gen_detectors(truths, start_time, number_steps, detection_probability, clutter_rate=clutter_rate)

# Run tracker
tracks = run_tracker(all_measurements, measurement_model, transition_model, detection_probability, death_probability, clutter_rate, start_truths, start_time)

Plotting the ground truths give the following plot.

In [3]:
from stonesoup.plotter import Plotterly
plotter = Plotterly()
plotter.plot_ground_truths(truths, [0, 2])
plotter.fig

Plotting the measurements (including clutter)

In [4]:
# Plot true detections and clutter.
plotter.plot_measurements(all_measurements, [0, 2])
plotter.fig

Now let us plot the tracks for a detection probability of 0.9.

In [5]:
from stonesoup.types.detection import TrueDetection
x_min, x_max, y_min, y_max = 0, 0, 0, 0

# Get bounds from the tracks
for track in tracks:
    for state in track:
        x_min = min([state.state_vector[0], x_min])
        x_max = max([state.state_vector[0], x_max])
        y_min = min([state.state_vector[2], y_min])
        y_max = max([state.state_vector[2], y_max])

# Get bounds from measurements
for measurement_set in all_measurements:
    for measurement in measurement_set:
        if type(measurement) == TrueDetection:
            x_min = min([measurement.state_vector[0], x_min])
            x_max = max([measurement.state_vector[0], x_max])
            y_min = min([measurement.state_vector[1], y_min])
            y_max = max([measurement.state_vector[1], y_max])

# Plot the tracks
plotter = Plotterly()
plotter.plot_ground_truths(truths, [0, 2])
plotter.plot_measurements(all_measurements, [0, 2])
plotter.plot_tracks(tracks, [0, 2], uncertainty=True)
plotter.fig.update_xaxes(range=[x_min-5, x_max+5])
plotter.fig.update_yaxes(range=[y_min-5, y_max+5])
plotter.fig

As observed and expected in the above plot, the filter can accurately differentiate between the different paths, newly birthed paths, and paths that have died.

With a high detection probability, the PHD filter is able to resolve the ground truths much more accurately. This is due to the posterior intensity function being more highly influenced by the posterior density, which propogates the estimate for the RFS $\mathbf{x}$ given the previous observations.

Now let us create another plot but instead with a lower detection probability. Let us choose a detection probability of $0.4$.

In [6]:
# Reducing detection probability
detection_probability = 0.4
clutter_rate = 3.0
all_measurements, measurement_model = gen_detectors(truths, start_time, number_steps, detection_probability, clutter_rate=clutter_rate)

# Run tracker
tracks = run_tracker(all_measurements, measurement_model, transition_model, detection_probability, death_probability, clutter_rate, start_truths, start_time)

# Get bounds from the tracks
for track in tracks:
    for state in track:
        x_min = min([state.state_vector[0], x_min])
        x_max = max([state.state_vector[0], x_max])
        y_min = min([state.state_vector[2], y_min])
        y_max = max([state.state_vector[2], y_max])

# Get bounds from measurements
for measurement_set in all_measurements:
    for measurement in measurement_set:
        if type(measurement) == TrueDetection:
            x_min = min([measurement.state_vector[0], x_min])
            x_max = max([measurement.state_vector[0], x_max])
            y_min = min([measurement.state_vector[1], y_min])
            y_max = max([measurement.state_vector[1], y_max])

# Plot the tracks
plotter_1 = Plotterly()
plotter_1.plot_ground_truths(truths, [0, 2])
plotter_1.plot_measurements(all_measurements, [0, 2])
plotter_1.plot_tracks(tracks, [0, 2], uncertainty=True)
plotter_1.fig.update_xaxes(range=[x_min-5, x_max+5])
plotter_1.fig.update_yaxes(range=[y_min-5, y_max+5])
plotter_1.fig

As observed in the above plot, the filter can still discern the ground truths to a reasonable. However, it cannot discriminate between new and old paths, and sometimes it can confuse the same ground as two different paths birthed at different times. This effect is pronounced if two targets are very close together.

This is due to the fact that a lower detection probability, $p_{D,k}(x)$ implies that the propagated posterior intensity at time $k$ becomes increasingly dependent on the predicted intensity, $v_{k|k-1}$. Therefore, the recursive posterior density does not get propogated with more weight.

Now, let us observe what happens for an extremely low detection probability.

In [14]:
# Reducing detection probability
detection_probability = 0.01
clutter_rate = 3.0
all_measurements, measurement_model = gen_detectors(truths, start_time, number_steps, detection_probability, clutter_rate=clutter_rate)

# Run tracker
tracks = run_tracker(all_measurements, measurement_model, transition_model, detection_probability, death_probability, clutter_rate, start_truths, start_time)

# Get bounds from the tracks
for track in tracks:
    for state in track:
        x_min = min([state.state_vector[0], x_min])
        x_max = max([state.state_vector[0], x_max])
        y_min = min([state.state_vector[2], y_min])
        y_max = max([state.state_vector[2], y_max])

# Get bounds from measurements
for measurement_set in all_measurements:
    for measurement in measurement_set:
        if type(measurement) == TrueDetection:
            x_min = min([measurement.state_vector[0], x_min])
            x_max = max([measurement.state_vector[0], x_max])
            y_min = min([measurement.state_vector[1], y_min])
            y_max = max([measurement.state_vector[1], y_max])

# Plot the tracks
plotter_2 = Plotterly()
plotter_2.plot_ground_truths(truths, [0, 2])
plotter_2.plot_measurements(all_measurements, [0, 2])
plotter_2.plot_tracks(tracks, [0, 2], uncertainty=True)
plotter_2.fig.update_xaxes(range=[x_min-5, x_max+5])
plotter_2.fig.update_yaxes(range=[y_min-5, y_max+5])
plotter_2.fig

As observed above, the filter can still detect the paths with a somewhat reasonable accuracy, but it cannot differentiate between newly birthed paths.

## References
[1] B. . -N. Vo and W. . -K. Ma, "The Gaussian Mixture Probability Hypothesis Density Filter," in IEEE Transactions on Signal Processing, vol. 54, no. 11, pp. 4091-4104, Nov. 2006, doi: 10.1109/TSP.2006.881190.

[2] https://stonesoup.readthedocs.io/en/v0.1b12/auto_tutorials/filters/GMPHDTutorial.html#sphx-glr-auto-tutorials-filters-gmphdtutorial-py